# RAG-Gym Local Model Demo

这个 Notebook 展示了如何使用配置好的本地 LLM 和 Embedding 模型运行 RAG-Gym 的一个简单 Demo。

In [2]:
import os
import sys
import json
import pandas as pd

# 添加项目根目录到系统路径
sys.path.append(os.path.abspath("."))

import rag_gym
from src.data_loader import KIQA
from rag_gym.config import config_local_llm, config_local_embedding

## 1. 模型配置确认
确认我们正在使用之前配置的本地端口。

In [ ]:
print(f"Local LLM Name: {config_local_llm['model_name']}")
print(f"Local LLM Base URL: {config_local_llm['base_url']}")
print(f"Local Embedding Name: {config_local_embedding['model_name']}")
print(f"Local Embedding Base URL: {config_local_embedding['base_url']}")

## 2. 初始化 Agent 和环境
我们使用 `RAGAgent` 作为示例，它会自动执行“检索 -> 生成”的过程。

In [3]:
agent_type = "rag"
llm_name = config_local_llm['model_name']
retriever_name = "BGE-M3"  # 对应本地 8001 端口的模型
corpus_name = "Wikipedia" # 默认使用维基百科语料库

# 初始化 Agent
agent = rag_gym.RAGAgent(
    llm_name = llm_name,
    api = True,
)

# 初始化环境
env = rag_gym.make(
    retriever_name = retriever_name,
    corpus_name = corpus_name,
    max_iter = 5,
    k = 3, # 每次检索 3 篇文档
    cache = True
)

Cloning the wikipedia corpus from Huggingface...
[In progress] Embedding the wikipedia corpus with the bge-m3 retriever...


Cloning into './corpus/wikipedia'...
fatal: unable to access 'https://huggingface.co/datasets/MedRAG/wikipedia/': Failed to connect to 127.0.0.1 port 7890 after 0 ms: Connection refused


FileNotFoundError: [Errno 2] No such file or directory: './corpus/wikipedia/chunk'

## 3. 加载一个测试样本

In [ ]:
dataset = KIQA("hotpotqa")
item = dataset[0] # 取第一个样本
question = item['question']
print(f"Question: {question}")
print(f"Reference Answer: {item['answer']}")

## 4. 运行交互流水线
手动模拟 Agent 在环境中的一步交互。

In [ ]:
# 重置环境
observation, info = env.reset(question=question)

print("--- Step 1: Agent Generating Action ---")
actions = agent.generate_action(state=observation, temperature=0.0)
action = actions[0]
print(f"Action Type: {action.type}")
print(f"Action String: {action.action_string}")

print("\n--- Step 2: Environment Executing Action ---")
observation, reward, terminated, truncated, info = env.step(action)

if action.type == "search":
    print("Retrieved Documents:")
    for doc in observation.history.history[-1]['content']:
        print(f"- {doc['title']}: {doc['content'][:200]}...")
elif action.type == "finish":
    print(f"Final Answer: {action.action_string}")

## 5. 完成剩余步骤
让 Agent 继续交互直到结束。

In [ ]:
max_steps = 3
curr_step = 1

while curr_step < max_steps:
    actions = agent.generate_action(state=observation, temperature=0.0)
    action = actions[0]
    print(f"\n--- Step {curr_step + 1}: Action ---\n{action.action_string}")
    
    observation, reward, terminated, truncated, info = env.step(action)
    if terminated:
        print("\nTask Terminated.")
        break
    curr_step += 1

print("\n--- Final Result ---")
print(f"History: {json.dumps(observation.return_as_json()['history'], indent=2, ensure_ascii=False)}")